# Validate the annotated datasets

This notebook provides code for validating the evaluation set .txt files and retrieving information about the anthropomorphic components.

1. Make sure that no sentence was annotated with conflicting annotations
2. Make sure that there are no duplicate sentences in a sentence
3. Make sure that the .txt files used to create the evaluation sets are well-formed - i.e, the IDs contain the database prefix (used to locate them in the dataframe) and that each row contains exactly seven tab-separated values.
4. Check that the annotations are correct - e.g. the positive set contains only ['p1','p2','p3'] scores, the negative set contains only ['n1','n2','n3'] scores, and the inconclusive set has only 'inc'.
5. Retrieving the anthropomorphic components
6. Retrieving the AI entity lemmas (i.e. without descriptors and modifiers)

In [1]:
import re
        
def get_sentences_dict(cat,score):   

    sentences_dict = {}
    duplicate_ids = []
    duplicate_sentence_pairs = []
    
    sentences = open(f"../data/evaluation_sentences_txt/{cat}_{score}.txt","r")
    
    for line in sentences.readlines():
        line = line.strip()
        line = line.split("\t")
        if len(line) == 0:
            break
        sent_id = line[0]
        sent_info = line[1:]

        # wellformedness checks
        if len(line) != 7:
            print(f"The row with the ID {sent_id} in {cat}_{score}.txt is not well-formed.")
        id_prefix = sent_id[:6]
        if not re.match(r"^[1-7]{1}_(arx|acl)_", id_prefix):
            print(f"The ID {sent_id} in {cat}_{score} is not well-formed.")
        
        if sent_id not in sentences_dict:
            if sent_info not in sentences_dict.values():
                sentences_dict[sent_id] = sent_info
            else: # the sentence appears twice with different IDs 
                other_id = [key for key in sentences_dict if sentences_dict[key] == sent][0]
                duplicate_sentence = (other_id,sent_id)
                duplicate_sentence_pairs.append(duplicate_sentence) 
        else: # the sentence appears twice with the same ID
            duplicate_ids.append(sent_id)

    return sentences_dict,duplicate_ids,duplicate_sentence_pairs

In [2]:
def check_duplicates(cat,score):

    response = "No duplicate utterances."
    print(f"Checking for duplicate entries in {cat}_{score}.txt...")

    sentences_dict = get_sentences_dict(cat,score)[0] # dict of ids and sentence info
    duplicate_ids = get_sentences_dict(cat,score)[1] # list of duplicate sentences with identical ids
    duplicate_sentence_pairs = get_sentences_dict(cat,score)[2] # list of duplicate sentences with different ids

    if duplicate_ids:
        response = f"Resolve duplicates in the {score} set!!!"
        print("The sentences with the following ids appear twice: ",duplicate_ids,
             f" in {cat}_{score}.txt")

    if duplicate_sentence_pairs:
        response = f"Resolve duplicates in the {score} set!!!"
        print("The following ID pairs refer to the same sentence: ",duplicate_sentence_pairs,
             f" in {cat}_{score}.txt")

    return response

def check_annotations(cat,score):

    annotations_dict = {}

    if cat == "noun_phrases" or cat == 'possessives':
        annotations = ['p']
    else:
        if score == 'positive':
            annotations = ['p1','p2','p3']
        elif score == 'negative':
            annotations = ['n1','n2','n3']
        elif score == 'inconclusive':
            annotations = ['inc']

    print(f"Checking annotations in {cat}_{score}.txt:")

    sentences_dict = get_sentences_dict(cat,score)[0]
    all_sentences_info = sentences_dict.values()

    for sent_id,sent_info in sentences_dict.items():
        if sent_info[5] not in annotations:
            print(f"Fix incorrect annotation {sent_info[5]} in the sentence with the ID {sent_id}")
        if sent_info[5] not in annotations_dict:
            annotations_dict[sent_info[5]] = 1
        else:
            annotations_dict[sent_info[5]] += 1
            
    num_sentences = sum([x for x in annotations_dict.values()])
    print(f"There are {num_sentences} sentences in {cat}_{score}.txt.")
    
    return annotations_dict

def pairwise_conflict_check(cat,score1,score2):

    print(f"Comparing {score1} cases and {score2} cases for the {cat} set...")

    conflicting_annotation = False

    dict1 = get_sentences_dict(cat,score1)[0]
    dict2 = get_sentences_dict(cat,score2)[0]

    for id1,sent in dict1.items():
        if id1 in dict2:
            conflicting_annotation = True
            print(f"The {score1} sentence with the ID ",id1,f" appears in the {score2} set with the same ID")
        elif sent in dict2.values():
            conflicting_annotation = True
            id2 = [key for keys in dict2.keys() if dict2[key] == sent][0]
            print(f"The {score1} sentence with the ID  ",id1,
                  f" appears in the {score2} set with the ID ",id2)

    return conflicting_annotation

def check_conflicting_annotations(cat,case,other_cases):

    response = "No conflicting annotations."

    for other_case in other_cases:

        conflicting_annotations = pairwise_conflict_check(cat,case,other_case)
        if conflicting_annotations:
            response = "Resolve conflicts before proceeding."
            print(f"Conflicting annotations in the {case} and {other_case} sets!!!")

    return response

In [3]:
import spacy
nlp = spacy.load("en_core_web_md")

def get_ai_phrases(cat,score,idx):

    ai_components = {}
    
    sentences_dict = get_sentences_dict(cat,score)[0]
    all_sentences_info = sentences_dict.values()
    
    for sent_info in all_sentences_info:
        if sent_info[idx] not in ai_components:
            ai_components[sent_info[idx]] = 1
        else:
            ai_components[sent_info[idx]] += 1

    return ai_components

def get_entities_or_components(cat,score,idx):

    components = {}
    
    sentences_dict = get_sentences_dict(cat,score)[0]
    all_sentences_info = sentences_dict.values()
    
    for sent_info in all_sentences_info:
        elements = sent_info[idx].split(",")
        for element in elements:
            if element not in components:
                components[element] = 1
            else:
                components[element] += 1

    return components

def get_and_display_anthro_words(cat,case,pos):

    if pos == 'verbs':
        pos_tag = 'VERB'
    elif pos == 'adjectives':
        pos_tag = 'ADJ'
    else:
        print("choose a different POS you pos!")
	
    anthro_words = []
    
    anthro_components = get_entities_or_components(cat,case,4)
    # get only the words that match the desired POS:
    for anthro_component in anthro_components:
        if len(anthro_component.split()) > 1:
            doc = nlp(anthro_component)
            verbs = [token.text for token in doc if token.pos_ == 'ADJ']
        else:
            words = anthro_component.split()
            anthro_words.extend(words)
    anthro_words = set(anthro_words)

    print(f"all {case} {pos} in {category_is}:")
    print(sorted(list(anthro_words)))
    print()

    return anthro_words

def get_phrase_mask_entity_triplets(cat,score):

    phrase_mask_entity_triplets = []
    
    sentences_dict = get_sentences_dict(cat,score)[0]
    
    for sent_id,sent_info in sentences_dict.items():
        phrase_mask_entity_triplets.append((sent_info[1],sent_info[2],sent_info[3]))

    return phrase_mask_entity_triplets

### Check sentences for each category

Perform checks on the set:
1. check that the file contains no duplicate sentences
2. check that the same sentence does not appear twice in two sets of the same category - for sets with more than one label
3. check that the annotations in a given file are correct

The categories are:
1. verb_subjects - sentences in which the AI entity is the subject of an anthropomorphic verb (nsubj)
2. verb_objects - sentences in which the AI entity is object of an anthropomorphic verb (pobj,dobj)
4. adjective_phrases - sentences in which the AI entity is part of an anthropomorphic adjectival phrase
5. noun_phrases - sentences in which the AI entity is part of an anthropomorphic noun phrase
6. possessives - sentences in which the AI entity is immediately followed by a possessive marker
7. comparisons - sentences in which the AI entity is being compared to humans explicitly

In [4]:
cases = ["positive","negative","inconclusive"]
#cases = ["inconclusive"]
category_is = "verb_subjects" # bring it to the runway

for case in cases:

    # check that the file contains no duplicate sentences
    check1 = check_duplicates(category_is,case)
    print(check1,'\n')
    
    # check that the same sentence does not appear twice in two sets of the same category
    # not applicable for noun_phrases, possessives (always positive) and comparisons (always inconclusive)
    other_cases = [other_case for other_case in cases if other_case != case]
    if other_cases:
        check2 = check_conflicting_annotations(category_is,case,other_cases)
        print(check2, '\n')

    # check that the annotations in a given file are correct (i.e. no negative annotations in the positive set)
    check3 = check_annotations(category_is,case)
    print(check3,'\n')

Checking for duplicate entries in verb_subjects_positive.txt...
No duplicate utterances. 

Comparing positive cases and negative cases for the verb_subjects set...
Comparing positive cases and inconclusive cases for the verb_subjects set...
No conflicting annotations. 

Checking annotations in verb_subjects_positive.txt:
There are 56 sentences in verb_subjects_positive.txt.
{'p3': 16, 'p2': 38, 'p1': 2} 

Checking for duplicate entries in verb_subjects_negative.txt...
No duplicate utterances. 

Comparing negative cases and positive cases for the verb_subjects set...
Comparing negative cases and inconclusive cases for the verb_subjects set...
No conflicting annotations. 

Checking annotations in verb_subjects_negative.txt:
There are 61 sentences in verb_subjects_negative.txt.
{'n2': 28, 'n1': 18, 'n3': 15} 

Checking for duplicate entries in verb_subjects_inconclusive.txt...
No duplicate utterances. 

Comparing inconclusive cases and positive cases for the verb_subjects set...
Comparing

### Retrieve information for manual review

1. retrieve the complete AI phrases and their count
2. retrieve the masks and their count
3. retrieve all of the AI entities and their count
4. retrieve all of the potentially (non-)anthropomorphic components and their count
5. retrieve the AI phrase, mask, entity triplets and their unique ID for manual review

In [48]:
#cases = ["positive","negative","inconclusive"]
cases = ["inconclusive"]
category_is = "comparisons" # bring it to the runway

check_AI_phrases = False
check_masks = False
check_entities = False
check_anthro_components = False
check_AI_triplets = True

for case in cases:

    print(f"handling {case} {category_is} cases")

    # retrieve the complete AI phrases and their count
    if check_AI_phrases == True:
        components = get_ai_phrases(category_is,case,1)
        sorted_list = sorted([(key,value) for key,value in components.items()])
        for item in sorted_list:
            print(item[1],": ",item[0])

    # retrieve the masks and their count
    if check_masks == True:
        components = get_ai_phrases(category_is,case,2)
        sorted_list = sorted([(key,value) for key,value in components.items()])
        for item in sorted_list:
            print(item[1],": ",item[0])
    
    # retrieve all of the AI entities and their count
    if check_entities == True:
        anthro_components = get_entities_or_components(category_is,case,3)
        sorted_anthro_list = sorted([(key,value) for key,value in anthro_components.items()])
        for item in sorted_anthro_list:
            print(item[1],": ",item[0])

    # retrieve all of the potentially (non-)anthropomorphic components and their count
    if check_anthro_components == True:
        anthro_components = get_entities_or_components(category_is,case,4)
        sorted_anthro_list = sorted([(key,value) for key,value in anthro_components.items()])
        for item in sorted_anthro_list:
            print(item[1],": ",item[0])

    # retrieve the AI phrase, mask, entity triplets and their unique ID for manual review
    if check_AI_triplets == True:
        phrase_mask_entity_triplets = get_phrase_mask_entity_triplets(category_is,case)
        for i,item in enumerate(phrase_mask_entity_triplets):
            print(i+1,item)

    print()

handling inconclusive comparisons cases
1 ('the best bidirectional model', 'bidirectional model', 'model')
2 ('a model', 'model', 'model')
3 ('GPT-3', 'GPT-3', 'GPT-3')
4 ('LLMs', 'LLMs', 'LLM')
5 ('the largest models we tested', 'models', 'model')
6 ('The large language model called ChatGPT', 'large language model', 'model')
7 ('any autonomous system', 'system', 'system')
8 ('the model', 'model', 'model')
9 ('the model', 'model', 'model')
10 ('a human-like reasoning architecture', 'architecture', 'architecture')
11 ('LLMs', 'LLMs', 'LLM')
12 ('ChatGPT', 'ChatGPT', 'ChatGPT')
13 ('larger and more advanced LLMs', 'LLMs', 'LLM')
14 ('ChatGPT', 'ChatGPT', 'ChatGPT')
15 ('a human-like question answering system', 'question answering system', 'system')
16 ('Robotics and AI', 'Robotics and AI', 'AI')
17 ('AI', 'AI', 'AI')
18 ('AI', 'AI', 'AI')
19 ('powerful AI systems', 'AI systems', 'system')
20 ('LLMs', 'LLMs', 'LLM')
21 ('LLMs', 'LLMs', 'LLM')
22 ('state-of-the-art LLMs', 'LLMs', 'LLM')
23

### Display verbs and adjectives for positive, negative, and inconclusive cases as well as intersections

In [56]:
positive_verbs = get_and_display_anthro_words("verb_subjects","positive","verbs")
negative_verbs = get_and_display_anthro_words("verb_subjects","negative","verbs")
inconclusive_verbs = get_and_display_anthro_words("verb_subjects","inconclusive","verbs")

all_verbs_intersect = sorted(list((positive_verbs & negative_verbs & inconclusive_verbs)))

just_pos_neg_v_intersect = sorted([x for x in list((positive_verbs & negative_verbs)) if x not in all_verbs_intersect])
just_pos_inc_v_intersect = sorted([x for x in list((positive_verbs & inconclusive_verbs)) if x not in all_verbs_intersect])
just_neg_inc_v_intersect = sorted([x for x in list((negative_verbs & inconclusive_verbs)) if x not in all_verbs_intersect])

print("verbs that are both positive and negative:")
for item in just_pos_neg_v_intersect:
    print(item)
print()
print("verbs that are both positive and inconclusive:")
for item in just_pos_inc_v_intersect:
    print(item)
print()
print("verbs that are both negative and inconclusive:")
for item in just_neg_inc_v_intersect:
    print(item)
print()
print("verbs that can be positive, negative or inconclusive:")
for item in all_verbs_intersect:
    print(item)

all positive verbs in verb_subjects:
['analyze', 'ask', 'believe', 'cheat', 'collaborate', 'coordinate', 'deduce', 'determine', 'distinguish', 'find', 'identify', 'infer', 'interpret', 'know', 'learn', 'memorize', 'overlook', 'prefer', 'reason', 'recall', 'recognize', 'reflect', 'remember', 'select', 'struggle', 'teach', 'think', 'understand']

all negative verbs in verb_subjects:
['accept', 'act', 'assume', 'carry', 'claim', 'consider', 'cover', 'demonstrate', 'establish', 'evolve', 'explain', 'find', 'generate', 'involve', 'learn', 'perform', 'produce', 'refrigerate', 'represent', 'require', 'retrieve', 'run', 'see', 'support']

all inconclusive verbs in verb_subjects:
['acquire', 'act', 'consider', 'construct', 'control', 'create', 'demonstrate', 'differentiate', 'dissect', 'fail', 'guess', 'identify', 'infer', 'interpret', 'learn', 'see', 'simulate', 'streamline', 'translate']

verbs that are both positive and negative:
find

verbs that are both positive and inconclusive:
identify


In [55]:
positive_adjectives = get_and_display_anthro_words("adjective_phrases","positive","adjectives")
negative_adjectives = get_and_display_anthro_words("adjective_phrases","negative","adjectives")
inconclusive_adjectives = get_and_display_anthro_words("adjective_phrases","inconclusive","adjectives")

all_adjectives_intersect = sorted(list((positive_adjectives & negative_adjectives & inconclusive_adjectives)))

just_pos_neg_a_intersect = sorted([x for x in list((positive_adjectives & negative_adjectives)) if x not in all_adjectives_intersect])
just_pos_inc_a_intersect = sorted([x for x in list((positive_adjectives & inconclusive_adjectives)) if x not in all_adj_intersect])
just_neg_inc_a_intersect = sorted([x for x in list((negative_adjectives & inconclusive_adjectives)) if x not in all_adj_intersect])

print("adjectives that are both positive and negative:")
for item in just_pos_neg_a_intersect:
    print(item)
print()
print("adjectives that are both positive and inconclusive:")
for item in just_pos_inc_a_intersect:
    print(item)
print()
print("adjectives that are both negative and inconclusive:")
for item in just_neg_inc_a_intersect:
    print(item)
print()
print("adjectives that can be positive, negative or inconclusive:")
for item in all_adjectives_intersect:
    print(item)

all positive adjectives in verb_subjects:
['attentive', 'autonomous', 'aware', 'blind', 'confident', 'confused', 'conscious', 'creative', 'deceptive', 'eager', 'forgetful', 'intelligent', 'malicious', 'manipulative', 'responsive', 'self-aware', 'sensitive', 'smart', 'thoughtful', 'value-aligned', 'vulnerable']

all negative adjectives in verb_subjects:
['blind', 'character-aware', 'communicative', 'concept-aware', 'context-aware', 'conversational', 'creative', 'dangerous', 'deep-learning', 'ethical', 'few-shot', 'image-blind', 'intelligent', 'materials-aware', 'multilingual', 'multimodal', 'original', 'responsible', 'risk-sensitive', 'robust', 'sensitive', 'smart', 'time-aware', 'time-sensitive', 'transparent', 'trustworthy', 'unreliable', 'unsafe', 'untrustworthy', 'user-friendly', 'vulnerable']

all inconclusive adjectives in verb_subjects:
['attentive', 'autonomous', 'collaborative', 'communicative', 'competitive', 'creative', 'dishonest', 'intelligent', 'responsible', 'sensitive', 